In [1]:
import sys
import os

import sys, os

# Ajouter en première position le dossier contenant ton module local
sys.path.insert(0, os.path.abspath(".."))

from backtesting.core import BacktestEngine

import pandas as pd 
import numpy as np

In [2]:
# We load our order book
orders_df_input = pd.read_excel(r"order_book.xlsx")
orders_df_input.drop(columns=['Unnamed: 0'], inplace=True)
# We load stock prices
df_stock_prices = pd.read_excel(r"StockPrices.xlsx")
# We format our df
df_stock_prices.set_index("Unnamed: 0", inplace=True)
df_stock_prices.index.name = None
df_stock_prices.head()


,ESCA,CMPVF,ESBA,HSON,WY,OPMXF,STR,NOTE,TXO,TDC,...,BICEF,KLBAY,ADI,ELP,HST,FECCF,NCHEY,SVTMF,HWM,NFE
2021-12-31,15.790000,78.150002,8.87,29.000000,41.180000,NaN,19.480000,9.92,NaN,42.470001,...,51.833000,8.145455,175.770004,5.63,17.389999,8.140,12.3,18.940001,31.830000,24.139999
2022-01-03,16.480000,78.150002,9.29,27.389999,40.570000,NaN,20.280001,9.92,NaN,43.919998,...,56.549999,8.109091,177.149994,5.51,17.660000,8.271,12.3,18.940001,32.330002,25.030001
2022-01-04,17.000000,78.150002,9.51,27.650000,41.090000,NaN,21.400000,9.91,NaN,43.799999,...,56.549999,8.363636,175.550003,5.45,17.910000,8.372,12.3,18.940001,33.529999,25.459999
2022-01-05,16.799999,78.150002,9.42,26.260000,40.070000,NaN,21.040001,9.93,NaN,42.610001,...,56.549999,8.363636,172.839996,5.29,17.379999,7.800,12.3,18.940001,33.320000,24.309999
2022-01-06,16.389999,78.150002,10.08,27.920000,40.389999,NaN,21.320000,9.93,NaN,42.840000,...,56.549999,8.363636,173.389999,5.20,17.350000,7.680,12.3,18.940001,33.650002,24.320000


In [3]:
bench_df_input = pd.read_excel(r"ETF_Bench.xlsx")
bench_df_input.set_index('Unnamed: 0', inplace=True)
bench_df_input.index.name = None
# bench_df_input.drop(columns=['Unnamed: 0'], inplace=True)
bench_df_input

,SPY,IWM,VTV,VUG,MTUM,USMV,QUAL,VYM,SPHQ,VLUE,...,XLV,XLF,XLY,XLP,XLE,XLI,XLB,XLU,XLRE,XLC
2015-07-28,209.330002,121.599998,83.739998,109.930000,73.730003,41.570000,64.470001,67.610001,23.370001,65.629997,...,76.129997,20.341187,78.870003,50.209999,70.470001,53.660000,45.509998,43.070000,NaN,NaN
2015-07-29,210.770004,121.940002,84.440002,110.519997,74.209999,41.849998,65.010002,68.150002,23.610001,66.239998,...,76.260002,20.544273,79.680000,50.380001,71.430000,54.320000,45.840000,43.230000,NaN,NaN
2015-07-30,210.820007,122.290001,84.480003,110.620003,74.139999,41.869999,65.220001,68.180000,23.629999,66.150002,...,76.150002,20.544273,79.970001,50.209999,70.930000,54.270000,46.070000,43.549999,NaN,NaN
2015-07-31,210.500000,122.959999,84.199997,110.550003,74.150002,41.939999,65.019997,68.029999,23.650000,65.910004,...,76.589996,20.479284,80.220001,50.310001,69.379997,54.220001,45.950001,43.990002,NaN,NaN
2015-08-03,209.789993,122.349998,83.930000,110.169998,74.070000,41.980000,64.760002,67.830002,23.570000,65.769997,...,76.529999,20.454914,79.949997,50.380001,67.919998,53.959999,45.590000,44.220001,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-18,627.580017,222.330002,177.660004,448.390015,239.070007,92.949997,184.029999,134.690002,71.370003,113.970001,...,131.839996,52.540001,222.169998,80.889999,85.959999,151.460007,90.220001,83.750000,41.840000,106.190002
2025-07-21,628.770020,221.429993,177.429993,450.000000,239.550003,92.970001,184.419998,134.610001,71.440002,114.029999,...,131.089996,52.389999,223.220001,80.879997,85.070000,150.660004,90.680000,84.019997,42.000000,107.610001
2025-07-22,628.859985,223.270004,178.869995,447.279999,237.559998,93.800003,184.830002,135.460007,71.699997,114.860001,...,133.529999,52.680000,225.479996,81.620003,85.470001,151.020004,91.930000,85.050003,42.730000,107.860001
2025-07-23,634.210022,226.679993,180.570007,450.399994,240.360001,93.970001,186.250000,136.660004,72.480003,116.510002,...,136.270004,53.099998,226.550003,81.650002,86.809998,153.729996,92.239998,84.410004,42.849998,108.449997


In [4]:
engine = BacktestEngine(orders_df_input, df_stock_prices, bench_df_input)
engine.run()

# Récupérer les outputs
print(engine.summary())  # Les métriques
# engine.plot_graphs()


c:\Users\cyril\backtesting_lib\backtesting\utils.py:137: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  condition_renfo_last_not_open = (~condition_open_position.shift(1).replace(np.nan, False)) & (df_volume_en_portefeuille * df_volume_en_portefeuille.shift(1) > 0) & (df_volume_flows * df_volume_en_portefeuille.shift(1) > 0)


{'sharpe_ratio': -2.28, 'sortino_ratio': -4.16, 'calmar_ratio': -8.47, 'max_cash_needed': 276959.02, 'total_profit': 62337.82, 'total_return': 0.2251, 'annualized_return': -0.3438, 'volatility': 0.1639, 'max_drawdown': 0.0406, 'average_transac_day': 107.5, 'max_nb_transac_day': 367, 'nb_transacs_total': 860, 'daily_portfolio_average_return': 0.0007, 'Hit_ratio': np.float64(0.558252427184466), 'Winner_median': np.float64(0.24821200498924623), 'Loser_median': np.float64(-0.18709372506271948), 'Avg_profit_per_ops': np.float64(0.18764227654933474), 'Median_holding_period': np.float64(1293.0), 'alpha': 0.000789, 'alpha_annualized': 0.198737, 'r_squared': 0.1011, 'residuals_std': 0.009789, 'n_observations': 1294}


In [5]:
engine.plot_graphs()